# LDA介绍

## 主题模型简介
主题模型（Topic Model）在机器学习和自然语言处理等领域是用来在一系列文档中发现抽象主题的一种统计模型。直观来讲，如果一篇文章有一个中心思想，那么一些特定词语会更频繁的出现。比方说，如果一篇文章是在讲狗的，那“狗”和“骨头”等词出现的频率会高些。如果一篇文章是在讲猫的，那“猫”和“鱼”等词出现的频率会高些。而有些词例如“这个”、“和”大概在两篇文章中出现的频率会大致相等。但真实的情况是，一篇文章通常包含多种主题，而且每个主题所占比例各不相同。因此，如果一篇文章10%和猫有关，90%和狗有关，那么和狗相关的关键字出现的次数大概会是和猫相关的关键字出现次数的9倍。一个主题模型试图用数学框架来体现文档的这种特点。主题模型自动分析每个文档，统计文档内的词语，根据统计的信息来断定当前文档含有哪些主题，以及每个主题所占的比例各为多少。主题模型最初是运用于自然语言处理相关方向，但目前以及延伸至例如生物信息学的其它领域。

下面我用一个实际的例子来说明主题模型的工作原理，下图是我们手头上有的一个语料库，一共有7篇文章

![](http://7xikew.com1.z0.glb.clouddn.com/topic_model1.png)

我们要将这些文档分配到三个主题，Topic1是关于信息技术的，Topic2是关于商业，Topic3是关于电影
![](http://7xikew.com1.z0.glb.clouddn.com/topic_model2.png)

经过主题模型分析后，我们会得到关于每篇文章的一个主题分布，主题分布告诉我们一篇文章上各个主题的占比是多少

![](http://7xikew.com1.z0.glb.clouddn.com/topic_model5.png)

比如黄颜色的文档属于技术的主题的概率远高于其他主题的概率；橙色文章位于主题1和主题2的中间，因此技术主题和商业主题的比例各占一半；而灰色文章位于三个主题的中心，因此三个主题的比例大致相等。



## 矩阵分解

我们还可以从矩阵分解的角度来理解LDA。具体来说，就是将文档-词频矩阵分解为文档-主题矩阵与主题-词矩阵的乘积。

![](http://7xikew.com1.z0.glb.clouddn.com/topic_model4.png)

图中，$M$代表文章数，$K$代表主题数，$V$代表词表大小。

在矩阵分解之前，文本挖掘领域在文本特征表示问题上所采用的主流方法是向量空间模型。然而，传统的基于向量空间模型的方法有一个致命缺陷，那就是特征稀疏问题。一方面，算法的效率会受到影响；另一方面，增加了存储上的开销。

作为改进，以LSA（潜在语义分析）为代表的矩阵分解方法被提出。LSA的革新之处在于打破了人们在文本表示上的思维定势，即认为文档是表示在词典空间上的。通过在文档和词之间引入一个语义维度，从而让文档和词在更低维的语义空间上建立起联系。语义是文档集合信息的一种浓缩，在LSA框架下，文档被表示为语义空间上的向量，向量值代表文档与语义的关联度；语义被表示为词典空间上的向量，向量值代表词和语义的关联度。

其基本原理是运用线性代数中的矩阵分解方法（奇异值分解，简称SVD）将文档-词频矩阵分解为两个维度更低的矩阵的乘积，从而获得原始矩阵的低秩近似（low-rank approximation）。这么做的好处是可以去除噪音，降低数据表示成本。

LSA的缺点是可解释性不强，它虽然能获得文档的语义表示，但不能解答文档集是如何产生的这一问题。针对于此，David M.Blei将概率模型引入到LSA框架中，提出LDA（潜在狄利克雷分配）算法。由于引入概率，所以LDA是一种生成式的矩阵分解方法（Generative Matrix Factorization）。与LSA相似的地方是，LDA也试图在文档->词的映射中引入一个中间变量，所不同的是他们把这一抽象概念称为主题。



## 基础知识介绍


### 多项式分布

多项式分布是文本处理领域最常用的分布之一，它是定义在一组离散随机变量上的分布，由一个和为1的参数向量表示，其概率分布数学形式为

$$
Multi(n_1,n_2,...,n_K|\boldsymbol{p},N)=\binom N {n_1n_2...n_K}\prod_{k=1}^K p_k^{n_k}
$$

其中$K$为所有可能结果数，$\mathbf{p}=(p_1,...,p_K)$，$p_k$表示一次独立实验中第$k$种结果发生的概率，$N$表示总观测次数（每次观测相互独立），$n_k$表示$N$次观测中第$k$种结果出现的次数，$N$和$n_k$的关系是$\sum_{i=1}^K n_i=N$。我们可以把$N$次观测的结果表示为一个向量$\mathbf{n}=\{n_1n_2...n_K\}$

$\binom N {n_1n_2...n_K}=\frac{N!}{n_1!n_2!...n_K!}$表示可能结果总数
理解多项式分布最简单的例子就是掷骰子。假设我们有一枚六面的骰子（$K=6$），当我们独立地投掷这枚骰子$N$次，6个面的总出现次数$\mathbf{n}=\{n_1,...,n_6\}$所构成的分布就服从一个多项式分布:
$$
Multi(n_1,n_2,...,n_6|\boldsymbol{p},N)=\binom N {n_1n_2...n_6}\prod_{k=1}^6 p_k^{n_k}
$$

### 多项式分布的可视化

前面提到，多项式分布可以由参数向量$\mathbf{p}$表示，现在我们来对它进行可视化分析。如下图所示，当$K=3$时，多项式分布可以表示为三角形区域中的一个点,这些点的坐标的和为1，这个三角形区域在数学上被称为单纯形（simplex）。

![](http://7xikew.com1.z0.glb.clouddn.com/topic_model6.png)



### 狄利克雷分布


狄利克雷分布是一组连续多变量概率分布，是Beta分布在多变量情况下的推广。为了纪念德国数学家约翰·彼得·古斯塔夫·勒热纳·狄利克雷（Peter Gustav Lejeune Dirichlet）而命名。狄利克雷分布常作为贝叶斯统计的先验概率。当狄利克雷分布维度趋向无限时，便成为狄利克雷过程（Dirichlet process）。
狄利克雷分布奠定了狄利克雷过程的基础，被广泛应用于自然语言处理特别是主题模型（topic model）的研究。

狄利克雷分布数学定义式为

$$Dir(\mathbf{p}|\alpha)=\frac{\Gamma(\sum_{k}\alpha_k)}{\prod_{k}\Gamma(\alpha _k)}\prod_{k}p_k^{\alpha_k-1}$$

其中$\alpha$是狄利克雷分布的参数。

### 狄利克雷分布的均值

在继续深入之前，先证明一个结论，狄利克雷分布的均值为

$$\mu_j=\frac{\alpha_j}{\alpha_0}$$
其中$\alpha_0=\sum_{k=1}^K \alpha_k$

证明：

$$
\begin{aligned}\mu_j&=\int_{ \boldsymbol{p}\in A} p_j Dir(\boldsymbol{p}|\alpha)d\boldsymbol{p}\\&=\frac{\Gamma(\alpha_0)}{\Gamma(\alpha_1)\cdots\Gamma(\alpha_K)}\int_{\boldsymbol{p}\in A}p_j^{(\alpha_j+1)-1}\prod_{k=1,k\neq j}^K p_k^{\alpha_k-1} d p_k d p_j \\&=\frac{\Gamma(\alpha_0)}{\Gamma(\alpha_1)\cdots\Gamma(\alpha_K)}\frac{\Gamma(\alpha_1)\cdots\Gamma(\alpha_{j-1})\Gamma(\alpha_j+1)\Gamma(\alpha_{j+1})\cdots\Gamma(\alpha_K)}{\Gamma(\alpha_0+1)}\\&=\frac{\Gamma(\alpha_0)\Gamma(\alpha_j+1)}{\Gamma(\alpha_0+1)\Gamma(\alpha_j)}\\&=\frac{\alpha_j}{\alpha_0}\end{aligned}
$$

其中$A$表示$\mathbf{p}$所在的单纯形。$\alpha_0$决定了狄利克雷分布的陡峭程度（有点类似于高斯分布中的方差），$\alpha_0$越小，分布就越平缓；$\alpha_0$越大，分布就越陡峭。


### 参数的取值对于狄利克雷分布形状的影响

还是以K=3为例，狄利克雷分布定义在多项式分布的单纯形上。下图为我们演示了当参数$\alpha$变化时狄利克雷分布的等高线分布图（颜色越红表示密度越大）：

![](http://7xikew.com1.z0.glb.clouddn.com/topic_model7.png)




当$\alpha=(1,1,1)$时，$p_k^{\alpha_k-1}=1$，退化为均匀分布；其概率密度如下所示
![](http://7xikew.com1.z0.glb.clouddn.com/dirichlet2.png)


当$\alpha=(0.1,0.1,0.1)$时，分布将变得异常稀疏，密度主要会集中在单纯形的角落和边界上，其余位置会变得异常扁平，如下图所示：
<img src="http://7xikew.com1.z0.glb.clouddn.com/dirichlet1.png" width=500>

当$\alpha=(10,0,10,10)$时，分布的均值向量是$m=(1/3,1/3,1/3)$，其概率密度的形状类似于凸起的山丘，其最高点就是均值
![](http://7xikew.com1.z0.glb.clouddn.com/dirichlet3.png)



限于篇幅，狄利克雷分布的很多细节没法讲清楚，后面我会专门写一篇关于狄利克雷分布的笔记。


### 狄利克雷分布与多项式分布的关系

既然Beta分布是二项式分布的共轭先验，而狄利克雷分布是Beta分布在多维空间上的推广，那么我们可以做出假设：狄利克雷分布是多项式分布的共轭先验。这意味着如果我们选择狄利克雷分布作为多项式分布参数的先验分布，那么它的后验分布也是一个狄利克雷分布（只是参数与先验分布的不同）。

接下来我们来证明这个结论。如果把贝叶斯公式给做个简化，则有
$$p(x|y)p(y)\propto p(y|x)$$

因此，我们只需要证明

$$ Multi(\mathbf{n}|N,\boldsymbol{p}) \cdot  Dir(\mathbf{p}|\alpha) \propto Dir(\mathbf{p}|\alpha') $$

证明：

$$ \begin{aligned}Multi(\mathbf{n}|N,\boldsymbol{p}) \cdot  Dir(\mathbf{p}|\alpha \mathbf{m}) &= \binom N {n_1n_2...n_K}\prod_{k=1}^K p_k^{n_k} \cdot \frac{\Gamma(\sum_{k}\alpha_k)}{\prod_{k}\Gamma(\alpha _k)}\prod_{k}p_k^{\alpha_k-1}\\&= \binom N {n_1n_2...n_K} \frac{\Gamma(\sum_{k=1}^K\alpha_k)}{\prod_{k=1}^K\Gamma(\alpha_k)} \prod_{k=1}^K p_k^{n_k+\alpha_k - 1}\\&\propto \prod_{k=1}^K p_k^{n_k+\alpha_k - 1}\end{aligned}$$

注意到等式右端与狄利克雷分布的形式相同，通过类比可以得出，参数$\mathbf{p}$的后验分布为

$$Dir(p|\alpha')=\frac{\Gamma(\sum_{k=1}^K\alpha_k+n_k)}{\prod_{k=1}^K\Gamma(\alpha_k+n_k)} \prod_{k=1}^K p_k^{n_k+\alpha_k - 1}$$

于是我们可以得出结论，狄利克雷分布是多项式分布的共轭先验，这个结论可以表达为

$$ Dir(p|\alpha_1,...,\alpha_k) + Multi(n_1,...,n_k|N,p)\to Dir(p|\alpha_1+n_1,...,\alpha_k+n_k)$$


## LDA工作原理

LDA算法框架可以用如下的概率图模型表示

![](http://7xikew.com1.z0.glb.clouddn.com/topic_model8.png)

下表给出了图中公式符号的含义


|符号|含义|
|:|:|
|$D$|文档数|
|$N$|一篇文档的单词数|
|$ K$|主题数|
|$V$|词表大小|
|$\alpha$|$\theta_d$的先验分布的超参数|
|$\eta$|$\beta_k$的先验分布的超参数|
|$\beta_k\sim DIR(\eta)$|第$k$个主题下所有单词的分布|
|$\theta_d \sim DIR(\alpha)$|文档$d$的主题分布|
|$W_{d,n}\sim Multi(\beta_{Z_{d,n}})$|文档$d$第$n$个词|
|$Z_{d,n}\sim Multi(\theta_d)$|$W_{d,n}$所对应主题|

此处为了简化模型，我们假设所有文章都包含的单词数均为$N$。LDA模型下，一篇文档的产生过程如下：


>for $k$ in $1,2,...,K$:  
&emsp;$\beta_k\sim DIR(\eta,...,\eta)$  
for $d$ in $1,2,...,D$:  
&emsp;$\theta_d \sim DIR(\alpha,...,\alpha)$  
&emsp;for $n$ in $1,2,...,N$:  
&emsp;&emsp;$Z_{d,n}\sim Multi(\theta_d)$  
&emsp;&emsp;$W_{d,n}\sim Multi(\beta_{Z_{d,n}})$

为了更直观的叙述这个过程，我截取《LDA数学八卦》中的一张示意图（符号有些差异，不过不影响我们理解这个过程）：

![](http://7xikew.com1.z0.glb.clouddn.com/topic_model9.png)

我们可以把$\beta_k$看作有$V$个面的骰子（同理把$\theta_d$看作有$K$个面的骰子），而它们都是从狄利克雷分布（对应图中左方的两个坛子）中随机抽样生成的，上面的坛子里装的是是doc-topic骰子，下面的坛子装的是topic-word骰子。在此设定下，LDA模型生成文档的过程描述如下：  
1）从下面的坛子中随机取出$K$个topic-word骰子  
2）从上面的坛子随机取出一个doc-topic骰子  
3）随机投掷doc-topic骰子得到主题$Z_{d,n}$  
4）从第1步取出的K个骰子中取出编号为$Z_{d,n}$的骰子，并进行投掷，得到词$W_{d,n}$


## Dirichlet-Multinomial共轭

我们观察LDA的概率图模型，发现LDA模型可以分解为两个物理过程：  

* $\alpha\to\theta_d\to z_{d,n}$

这个过程对应于生成第$d$篇文档的时候，从上面的坛子中随机抽取一个doc-topic骰子$\theta_d$，再用这个骰子进行投掷，得到第$n$个词的topic编号$z_{d,n}$

* $\eta\to\beta_k\to w_{d,n}|z_{d,n}=k$

这个过程对应于在生成第$d$篇文档的第$n$个词的时候，先从下面的坛子中取出编号为$z_{d,n}$的topic-word骰子，再进行投掷得到$w_{d,n}$

注意到$\theta_d,\beta_k$满足Dirichlet分布，而$z_{d,n},w_{d,n}$服从Multinomial分布，根据上一节推导的结论可知，$\theta_d$和$\beta_k$的后验分布也是Dirichlet分布，这个性质称为Dirichlet-Multinomial共轭。



利用该性质，我们可以得出$\theta_d$和$\beta_k$的后验分布分别为：

$$\theta_d\sim Dir(\alpha+N_1^{(d)}, ..., \alpha+N_K^{(d)})$$

其中$N_k^{(d)}$表示第$d$篇文档中属于主题$k$的单词数，即$N_k^{(d)}=\sum_{n=1}^N \mathbb{1}(z_{d,n}=k)$  

其中$\theta_{d,k}$的参数估计为

$$\theta^*_{d,k}=\mathbb{E}[\theta_{d,k}]=\frac{\alpha+N_k^{(d)}}{K\alpha+\sum_{k=1}^K N_k^{(d)}}$$


$$ \beta_k \sim Dir(\eta+N_1^{(k)},...,\eta+N_V^{(k)}) $$

其中$N_v^{(k)}$表示第$v$个词在主题$k$中的出现次数

那么$\beta_{k,v}$的参数估计为

$$\beta^*_{k,v}=\mathbb{E}[\beta_{k,v}]=\frac{\eta+N_v^{(k)}}{V\eta+\sum_{v=1}^V N_v^{(k)}}$$

如此一来，LDA模型一共有$D+K$个Dirichlet-Multinomial共轭链